<a href="https://colab.research.google.com/github/MestDash/PID/blob/main/preprocess_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [15]:
df = pd.read_excel("ds_FLOWSOM_29092021.xlsx")
df = df.dropna(subset=["IUIS"])
df.shape

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


(399, 89)

In [ ]:
# In case I need classes as numbers
classes = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}
for i, row in df.iterrows():
  if str(row["IUIS"]) == "No arguments for lymphoid-PID":
    df.at[i, "IUIS"] = 0
    df.at[i, "IUIS extended"] = 0
  else:
    iuis = str(row["IUIS"])
    index = iuis.find(":")
    pid = iuis[:index]
    df.at[i, "IUIS"] = classes[pid]

In [ ]:
# IUIS extended
for i, row in df.iterrows():
  if str(row["IUIS extended"]) == "Unclassified":
    df.at[i, "IUIS extended"] = 0
  else:
    subclass = str(row["IUIS extended"])
    df.at[i, "IUIS extended"] = int(subclass[0])

In [16]:
label_columns = ['IgG (normalized against RV)', 'IGG2 (normalized against RV)',
                 'IGG3 (normalized against RV)','IgM (normalized against RV)',
                 'IGA (normalized against RV)']
df[label_columns] = df[label_columns].fillna('NA')
replacement_dict = {0: 'normal', -1: 'lower', 1: 'higher'}
df[label_columns] = df[label_columns].replace(replacement_dict)

In [17]:
df['Humoral or cellular (B or T-cell mediated)'] = df['Humoral or cellular (B or T-cell mediated)'].str.strip()

In [18]:
to_drop = ['Age (d)', 'labels: lymphoid PID vs HC vc DC vs non-lymphoid PID', 'Subgroep labels', 'Diagnose PID', 'Detailed diagnosis',
           'Humoral or cellular (B or T-cell mediated)', 'Deviations seen in current PIDOT routine appraoch (MG + comparison EuroFlow reference range)',
           'Opmerking', 'Date of birth ', 'Date of sample', 'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73',
           'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81',
           'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85', 'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88']
df = df.drop(columns=to_drop)

In [19]:
df["% lympho"] = df["% lympho"] /100
df = df.rename(columns={" lympho/µL": "lympho/µL"})
df = df.rename(columns={"EM CD8+ T-cells/µL ": 'EM CD8+ T-cells/µL'})

In [20]:
df.head(5)

,IUIS,IUIS extended,PCODE,M:F,Age (Y),WBC,Lymfo/µL (Sysmex),IgG,IGG2,IGG3,...,% Effector TD CD4+ T-cells (on CD4 T),% CD8+ T-cells (on T),% naieve CD8+ T-cells (on CD8 T),% CM CD8+ T-cells (on CD8 T),% EM CD8+ T-cells (on CD8 T),% Effector TD 27+ CD8+ T-cells (on CD8 T),% Effector TD CD8+ T-cells (on CD8 T),% DNT TCRgd-T-cells (on T),% TCRgd+ T-cells (on T),% NK cells (on lymfo)
0,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P001,F,12.250000,8720,2280,6,2.07,0.178,...,0.004,0.30,0.67,0.27,0.015,0.031,0.010,0.020,0.060,0.070
1,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P002,M,15.083333,6780,1940,5.9,1.63,0.175,...,0.001,0.27,0.57,0.35,0.031,0.044,0.005,0.013,0.121,0.081
2,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P003,F,17.250000,5950,2040,11.8,3.53,0.253,...,0.000,0.34,0.52,0.37,0.035,0.075,0.013,NaN,0.076,0.089
3,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P004,M,15.750000,3910,1350,10,2.46,0.929,...,0.000,0.43,0.43,0.42,0.141,0.088,0.041,0.025,0.018,0.047
4,III: Predominantly antibody defciencies,2. Severe Reduction in at Least 2 Serum Immuno...,P005,M,15.000000,9200,1860,12.6,2.79,1.27,...,0.020,0.33,0.45,0.01,0.013,0.443,0.080,0.018,0.260,0.153
